# Relation Extraction Model

This notebook conducts relation extraction on the annotated psychedelic research papers. It takes in binary spacey files with the individual labels defined. These have been converted from json format into the necessary spacy format prior to running the relation extraction model using the spacey_file_converter.py. 

To run the script, the GPU needs to be activated by clicking on 'Edit' ==> 'Notebook settings' ==> change hardware to 'GPU'. 

## Install Dependencies

In [ ]:
!pip install -U spacy-nightly --pre

In [ ]:
!pip install -U pip setuptools wheel

### Clone the Repository

We're using Spacy's new 'project' feature which allows you to clone their repository for a specific task and use it as a template for your model. This repository contains the folder architecture necessary for conducing relation extraction. 

In [ ]:
!python -m spacy project clone tutorials/rel_component

✔ Cloned 'tutorials/rel_component' from explosion/projects
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


### Install transformer pipeline

In [ ]:
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_trf-3.2.0/en_core_web_trf-3.2.0-py3-none-any.whl (460.2 MB)
  Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.12.5
    Uninstalling transformers-4.12.5:
      Successfully uninstalled transformers-4.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.1.3 requires transformers<4.13.0,>=3.4.0, but you have transformers 4.15.0 

### Change the working directory

In [ ]:
cd rel_component

/content/rel_component


## Train the sciBERT model 

We train the model on the GPU using Spacy's 'train_gpu' script, which is found in the cloned repository. Prior to running this, the train, test, and dev data has to be added into the paths and parameters need to be set. We've ammended some of the parameters so that the model is running with SciBERT, and with a max-lenth of tokens between entities set to 50.

In [ ]:
!spacy project run train_gpu 


================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train data/train.spacy --paths.dev data/dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-01-03 05:33:15,496] [INFO] Set up nlp object from config
[2022-01-03 05:33:15,507] [INFO] Pipeline: ['transformer', 'relation_extractor']
[2022-01-03 05:33:15,512] [INFO] Created vocabulary
[2022-01-03 05:33:15,514] [INFO] Finished initializing nlp object
Downloading: 100% 385/385 [00:00<00:00, 374kB/s]
Downloading: 100% 223k/223k [00:00<00:00, 700kB/s]
Downloading: 100% 422M/422M [00:10<00:00, 42.2MB/s]
Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias',

## Evaluate the Model 

In [ ]:
cd /content/drive/MyDrive/rel_component

/content/drive/MyDrive/rel_component


In [ ]:
!spacy project run evaluate

## Compare with a non-transformer tok2vec model 

Our trained model is quite disappointing in the relation extraction task, but to see what difference the transformer achitecture is making, we will also train a tok2vec model and see how well this model is able to perform. 

In [ ]:
!spacy project run train_cpu 
!spacy project run evaluate


================================= train_cpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_tok2vec.cfg --output training --paths.train data/train.spacy --paths.dev data/dev.spacy -c ./scripts/custom_functions.py
ℹ Saving to output directory: training
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-01-03 06:04:44,372] [INFO] Set up nlp object from config
[2022-01-03 06:04:44,384] [INFO] Pipeline: ['tok2vec', 'relation_extractor']
[2022-01-03 06:04:44,390] [INFO] Created vocabulary
[2022-01-03 06:04:44,392] [INFO] Finished initializing nlp object
[2022-01-03 06:04:46,863] [INFO] Initialized pipeline components: ['tok2vec', 'relation_extractor']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'relation_extractor']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC 

The tok2vec model is considerably worse than the transformer model, only being able to classify relations with an accuracy of around 18%. This tells us that we need to rethink some of our relationships, and perhaps simplify from classifying between 5 relationships to instead only classify between 2 or 3. These are ammendments which will be considered in the evaluation of the pipeline. 